# Import Packages

In [1]:
# built-in
import os.path

# third-party (pip install required)
import numpy as np
import pandas as pd
from pymodulon.compare import compare_ica
from pymodulon.core import IcaData
from pymodulon.io import load_json_model, save_to_json
from pymodulon.util import explained_variance
from tqdm.notebook import tqdm

In [2]:
# Enter the location of your data here
data_dir = '../../data/precise1k/'
annotation_dir = '../../data/annotation'

# QC Tables

## Metadata

Your metadata file will probably have a lot of columns, most of which you may not care about. Feel free to save a secondary copy of your metadata file with only columns that seem relevant to you. The two most important columns are:
1. `project`
2. `condition`

Make sure that these columns exist in your metadata file

In [3]:
df_metadata = pd.read_csv(os.path.join(data_dir,'metadata_qc.csv'),index_col=0)
df_metadata[['project','condition']].head()

,project,condition
Experiment,,
p1k_00001,control,wt_glc
p1k_00002,control,wt_glc
p1k_00003,fur,wt_dpd
p1k_00004,fur,wt_dpd
p1k_00005,fur,wt_fe


In [4]:
print(df_metadata.project.notnull().all())
print(df_metadata.condition.notnull().all())

True
True


## TRN

Each row of the TRN file represents a regulatory interaction.  
**Your TRN file must have the following columns:**
1. `regulator` - Name of regulator (`/` or `+` characters will be converted to `;`)
1. `gene_id` - Locus tag of gene being regulated

The following columns are optional, but are helpful to have:
1. `regulator_id` - Locus tag of regulator
1. `gene_name` - Name of gene (can automatically update this using `name2num`)
1. `direction` - Direction of regulation ('+' for activation, '-' for repression, '?' or NaN for unknown)
1. `evidence` - Evidence of regulation (e.g. ChIP-exo, qRT-PCR, SELEX, Motif search)
1. `PMID` - Reference for regulation

You may add any other columns that could help you. TRNs may be saved as either CSV or TSV files. See below for an example:

In [5]:
df_trn = pd.read_csv(os.path.join(annotation_dir,'TRN.csv'))
df_trn.head()

,regulator,gene_name,gene_id,effect,evidence,source
0,AccB,accB,b3255,-,0.0,RegulonDB_TF
1,RpoD,pheU,b4134,+,0.0,RegulonDB_sigma
2,RpoD,pheP,b0576,+,0.0,RegulonDB_sigma
3,RpoD,pheL,b2598,+,0.0,RegulonDB_sigma
4,RpoD,pheA,b2599,+,0.0,RegulonDB_sigma


The `regulator` and `gene_id` must be filled in for each row

In [6]:
print(df_trn.regulator.notnull().all())
print(df_trn.gene_id.notnull().all())

True
True


# Create IcaData

In [7]:
ica_data = IcaData(
    M=os.path.join(data_dir,'M.csv'),
    A=os.path.join(data_dir,'A.csv'),
    gene_table=os.path.join(annotation_dir,'gene_info.csv'),
    sample_table=os.path.join(data_dir,'metadata_qc.csv'),
    trn=os.path.join(annotation_dir,'TRN.csv'),
    optimize_cutoff=True
)

# TRN Enrichment
Use `compute_trn_enrichment` to automatically check for Regulatory iModulons. The more complete your TRN, the more regulatory iModulons you'll find.

For this automated step, use strict cutoff for evidence required (can loosen later during manual annotation)

In [11]:
ica_data.compute_trn_enrichment(max_regs=2, evidence=[1, 2], save=True)

/Users/cam/.virtualenvs/ica/lib/python3.10/site-packages/pymodulon/core.py:610: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  keep_cols = self.imodulon_table.loc[


,imodulon,regulator,pvalue,qvalue,precision,recall,f1score,TP,regulon_size,imodulon_size,n_regs
0,0,AllR/AraC,6.212559e-32,2.231178e-27,0.700000,0.736842,0.717949,14.0,19.0,20.0,2.0
1,0,AllR/Crp,2.008180e-19,3.606090e-15,0.900000,0.059801,0.112150,18.0,301.0,20.0,2.0
2,0,AllR,5.549280e-16,6.643229e-12,0.350000,0.777778,0.482759,7.0,9.0,20.0,1.0
3,0,AraC,1.844808e-15,1.325088e-11,0.350000,0.700000,0.466667,7.0,10.0,20.0,1.0
4,0,AraC+Crp,1.844808e-15,1.325088e-11,0.350000,0.700000,0.466667,7.0,10.0,20.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1089,170,ArcA/Crp,5.116862e-10,6.342086e-07,0.303371,0.078717,0.125000,27.0,343.0,89.0,2.0
1090,170,BetI/HypT,5.633668e-09,6.749885e-06,0.067416,0.666667,0.122449,6.0,9.0,89.0,2.0
1091,176,FliZ+GadE,6.430467e-10,8.398655e-06,0.444444,0.500000,0.470588,4.0,8.0,9.0,2.0
1092,176,FliZ/GadE-RcsB,7.014272e-10,8.398655e-06,0.555556,0.192308,0.285714,5.0,26.0,9.0,2.0


# Single-Gene-Dominant iModulons

In [8]:
sg_imods = ica_data.find_single_gene_imodulons(save=True)

In [9]:
for sg_mod in sg_imods:
    sg_name = ica_data.view_imodulon(sg_mod).sort_values(
        by='gene_weight', ascending=False).iloc[0, :]['gene_name']
    ica_data.rename_imodulons({sg_mod:f'{sg_name}'})

# iModulon Names

Convenient to string-ify all names so as not to have a mix of strings and ints in the index

In [10]:
rename_dict = {}
for im_name in ica_data.imodulon_names:
    if isinstance(im_name, int):
        rename_dict[im_name] = str(im_name)
ica_data.rename_imodulons(rename_dict)

# Explained Variance

In [13]:
ica_data.X = pd.read_csv(os.path.join(data_dir, 'log_tpm_norm_qc.csv'), index_col=0)

In [14]:
exp_vars = [explained_variance(ica_data, imodulons=[imod]) for imod in ica_data.imodulon_names]

In [15]:
ica_data.imodulon_table['exp_var'] = exp_vars

In [16]:
ica_data._x = None

# iModulon Size

In [17]:
ica_data.imodulon_table['imodulon_size'] = [ica_data.view_imodulon(imod).shape[0]
                                            for imod in ica_data.imodulon_names]

# PRECISE and PRECISE 2.0 Comparison

## Load Previous Datasets

In [18]:
precise1 = load_json_model('../../data/precise/precise.json.gz')
precise2 = load_json_model('../../data/precise2/precise2.json.gz')

## Run Correlation Comparisons

In [20]:
match_rows = []
for match in compare_ica(precise1.M, ica_data.M, method='pearson')[0]:
    match_rows.append({
        'dataset': 'PRECISE',
        'iM': match[0],
        'p1k_im': match[1],
        'method': 'pearson',
        'corr': match[2]
    })
for match in compare_ica(precise1.M, ica_data.M, method='spearman')[0]:
    match_rows.append({
        'dataset': 'PRECISE',
        'iM': match[0],
        'p1k_im': match[1],
        'method': 'spearman',
        'corr': match[2]
    })
for match in compare_ica(precise2.M, ica_data.M, method='pearson')[0]:
    match_rows.append({
        'dataset': 'PRECISE 2.0',
        'iM': match[0],
        'p1k_im': match[1],
        'method': 'pearson',
        'corr': match[2]
    })
for match in compare_ica(precise2.M, ica_data.M, method='spearman')[0]:
    match_rows.append({
        'dataset': 'PRECISE 2.0',
        'iM': match[0],
        'p1k_im': match[1],
        'method': 'spearman',
        'corr': match[2]
    })
    
match_df = pd.DataFrame(match_rows)

In [21]:
for im, im_row in ica_data.imodulon_table.iterrows():
    im_match = match_df[match_df['p1k_im'] == im]
    if not im_match.empty:
        for dataset, dataset_match_df in im_match.groupby('dataset'):
            max_corr = np.argmax(dataset_match_df['corr'])
            max_corr_im = dataset_match_df.iloc[max_corr]['iM']
            max_corr_im_df = dataset_match_df[dataset_match_df['iM'] == max_corr_im]
            for _, max_corr_row in max_corr_im_df.iterrows():
                ica_data.imodulon_table.loc[im, max_corr_row['dataset']] = max_corr_row['iM']
                ica_data.imodulon_table.loc[im, f"{max_corr_row['dataset']}_{max_corr_row['method']}"] = max_corr_row['corr']

# Re-Save M/A Matrices and iModulon Table

In [60]:
ica_data.imodulon_table.to_csv(os.path.join(data_dir,'imodulon_table.csv'))
ica_data.A.to_csv(os.path.join(data_dir,'A.csv'))
ica_data.M.to_csv(os.path.join(data_dir,'M.csv'))

# Save IcaData Object

This will save your iModulon table, your thresholds, and any other information stored in the ica_data object.

In [61]:
save_to_json(ica_data, os.path.join(data_dir,'precise1k'), compress=True)